#Montar o drive para acesar base de dados

In [0]:
# Solicita autorização para acesso ao seu drive
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import os
os.chdir("/content/drive/My Drive") #MAPEAR PARA O LOCAL NO DRIVE ONDE SE ENCONTRA ESSE ARQUIVO

In [0]:
!ls #Ver arquivos contidos no diretório mapeado! verifique se sua base está listada!

In [0]:
"""### Importacao de bibliotecas"""
import numpy as np
import pandas as pd

# Passo 1. Carregar base de dados

In [0]:
#Carregar a base e visualizar primeiras linhas
data = pd.read_csv('LifeExpectancy.csv')
#Visualize a base pelo Explorador de Variáveis

data.head()

In [0]:
#dimnesões da base
data.shape

In [0]:
data.describe()

In [0]:
#Missing Data
columns = ['Year', 'LifeExpectancy','AdultMortality','InfantDeaths','Alcohol','PercentageExpenditure','HepatitisB','Measles',' BMI ','under-fivedeaths ','Polio','TotalExpenditure','Diphtheria','HIV/AIDS','GDP','Population','thinness1-19years','thinness5-9years','IncomeCompositionOfResources','Schooling']

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2, weights="uniform")
for i in columns:
    data[i] = imputer.fit_transform(data[[i]])

data.describe()

In [0]:
# OneHotEncoding
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder()
data.Status = onehotencoder.fit_transform(data[['Status']]).toarray()

data.Status.value_counts()

#Passo 2. Separar base em treino e teste

In [0]:
np.random.seed(0) #semente inicial
nlinhas = data.shape[0]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.loc[:,data.columns != 'Lifeexpectancy'], data['LifeExpectancy'], test_size=0.3)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [0]:
"""## Análise exploratória"""
import seaborn as sns
corr = data.corr()
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)

# Passo 3. Normalizar Dados

In [0]:
"""## **Normalizando os Dados**"""
from sklearn import preprocessing
preprocessParams = preprocessing.StandardScaler().fit(X_train)
X_train_normalized = preprocessParams.transform(X_train)
X_test_normalized = preprocessParams.transform(X_test)

X_train_normalized[0:2, :]

# Passo 4. Treinar Rede Neural

In [0]:
#Utilizar uma versão do tensorflow anterior a 2.0
%tensorflow_version 1.x

In [0]:
"""# Rede Neural"""
from keras import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

RN = Sequential()
RN.add(Dense(10,input_shape = X_train_normalized.shape[1:],activation = 'sigmoid'))
RN.add(Dense(1))
RN.summary()

In [0]:
# treinamento
from keras.utils import to_categorical
RN.compile(optimizer = 'sgd', loss = 'mean_squared_error', metrics = ['mape']) 
trainedRN = RN.fit(X_train_normalized,y_train, epochs = 100, verbose = 1)

# Passo 5. Avaliação da Rede Neural

In [0]:
# avaliação
score = trainedRN.model.evaluate(X_test_normalized, y_test, verbose = 0)
print('Test score:', score[0])
print('Test mape:', score[1], '%')

In [0]:
from draw_neural_net import PlotHistory
PlotHistory(trainedRN.history)

# Passo 6. Fazer novas previsões

In [0]:
from sklearn.metrics import confusion_matrix
y_test_predicted = RN.predict(X_test_normalized)
y_test_predicted

In [0]:
# Avaliação gráfica
from matplotlib import pyplot
results = pd.DataFrame(y_test_predicted, columns = ['previsto'])
results['real'] = np.array(y_test)
results.plot(); #todas as previsões

In [0]:
results.iloc[1:100,:].plot() # somente as 100 primeiras